This section begins a new major category: **Statistical Analysis**, starting with **1. Descriptive Statistics**.

Descriptive statistics provide a fundamental framework for understanding data through quantitative summaries and visualizations. This analysis includes calculating measures of central tendency (mean, median, mode), measures of dispersion (variance, standard deviation, range), and measures of shape (skewness, kurtosis), along with analyzing the distribution of categorical variables.

Based on [`piplist.txt`](./README.md) output, we have `pandas` (excellent for summary statistics), `numpy` (for numerical operations), `seaborn`, and `matplotlib` (both for visualization of distributions), which are the core tools for descriptive analysis.

**Step 1: Generate Sample Data for Descriptive Statistics**

We'll create a dummy dataset containing a mix of numerical and categorical columns to demonstrate various descriptive statistics.

In a new Excel cell, enter `=PY` and paste the following code, then press **Ctrl+Enter**:

In [ ]:
# Generate dummy data for Descriptive Statistics
import pandas as pd
import numpy as np
from faker import Faker
import random

fake = Faker()

num_records = 1200

data = {
    'EmployeeID': range(101, 101 + num_records),
    'Age': [random.randint(22, 65) if random.random() > 0.02 else np.nan for _ in range(num_records)], # Age with some missing
    'AnnualSalary': [round(random.uniform(40000, 150000), 2) for _ in range(num_records)],
    'YearsAtCompany': [random.randint(1, 20) if random.random() > 0.01 else np.nan for _ in range(num_records)], # Years with some missing
    'Department': [random.choice(['Sales', 'Marketing', 'IT', 'Finance', 'HR', 'Engineering', 'Operations']) for _ in range(num_records)],
    'PerformanceRating': [random.choice([1, 2, 3, 4, 5, 5, 4, 3, 2, 1, np.nan]) for _ in range(num_records)], # Rating with missing, skewed towards middle/high
    'IsManager': [random.choice([True, False, False, False, None]) for _ in range(num_records)], # Boolean with missing
    'SatisfactionScore': [round(random.uniform(2.5, 5.0), 1) if random.random() > 0.03 else np.nan for _ in range(num_records)] # Score out of 5
}

df_desc = pd.DataFrame(data)

# Introduce a couple of extreme outliers in Salary
df_desc.loc[random.sample(range(num_records), 2), 'AnnualSalary'] = [500000, 750000]

df_desc # Output the DataFrame

**Explanation:**

*   This code generates a DataFrame `df_desc` simulating employee data.
*   It includes numerical columns (`Age`, `AnnualSalary`, `YearsAtCompany`, `SatisfactionScore`), categorical columns (`Department`), an ordinal-like numerical column (`PerformanceRating`), and a boolean column (`IsManager`).
*   Missing values (`np.nan`, `None`) are introduced, as well as a few high outliers in `AnnualSalary`.
*   The result, `df_desc`, will be spilled into your Excel sheet. Let's assume this data is placed in a range or Table named `EmployeeData`.

**Step 2: Calculate and Visualize Descriptive Statistics**

Now, we'll load this dummy data and calculate standard descriptive statistics using `pandas`, then visualize the distributions of a numerical and a categorical column using `seaborn`/`matplotlib`.

In a **new** Excel cell, enter `=PY` and paste the following code. Replace `"EmployeeData"` with the actual name of the Excel range/Table where your dummy data is. Press **Ctrl+Enter**.

In [ ]:
# Calculate and visualize descriptive statistics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data from Excel
# IMPORTANT: Replace "EmployeeData" with the actual name of your Excel range or Table
df = xl("EmployeeData[#All]", headers=True)

# Ensure appropriate data types (xl often reads as object)
# Coerce errors to NaN for numeric columns that might have non-numeric strings if not clean
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['AnnualSalary'] = pd.to_numeric(df['AnnualSalary'], errors='coerce')
df['YearsAtCompany'] = pd.to_numeric(df['YearsAtCompany'], errors='coerce')
df['PerformanceRating'] = pd.to_numeric(df['PerformanceRating'], errors='coerce') # Can be treated as numeric or ordinal categorical
df['SatisfactionScore'] = pd.to_numeric(df['SatisfactionScore'], errors='coerce')
# Handle None/NaN in 'IsManager' before converting to boolean dtype
# Use replace to explicitly change None to boolean False, then convert to nullable boolean
df['IsManager'] = df['IsManager'].replace({0: False}).astype('boolean')


# --- 1. Overall Descriptive Summary ---
# Use describe() for both numeric and categorical columns
descriptive_summary_numeric = df.describe()
descriptive_summary_all = df.describe(include='all')


# --- 2. Specific Metrics (Examples) ---
mean_salary = df['AnnualSalary'].mean()
median_years = df['YearsAtCompany'].median()
mode_department = df['Department'].mode()[0] if not df['Department'].mode().empty else 'N/A' # mode() can return multiple if tie
salary_std_dev = df['AnnualSalary'].std()
salary_variance = df['AnnualSalary'].var()
salary_range = df['AnnualSalary'].max() - df['AnnualSalary'].min() if df['AnnualSalary'].count() > 1 else np.nan
age_q1 = df['Age'].quantile(0.25)
age_q3 = df['Age'].quantile(0.75)


# Structure specific metrics into a DataFrame
specific_metrics = pd.DataFrame({
    'Metric': [
        'Mean Annual Salary', 'Median Years at Company', 'Mode Department',
        'Annual Salary Std Dev', 'Annual Salary Variance', 'Annual Salary Range',
        'Age Q1 (25th Percentile)', 'Age Q3 (75th Percentile)'
        ],
    'Value': [
        mean_salary, median_years, mode_department,
        salary_std_dev, salary_variance, salary_range,
        age_q1, age_q3
        ]
})


# --- 3. Frequency Counts (for categorical/discrete data) ---
department_counts = df['Department'].value_counts(dropna=False).reset_index()
department_counts.columns = ['Department', 'Count']

performance_rating_counts = df['PerformanceRating'].value_counts(dropna=False).reset_index()
performance_rating_counts.columns = ['PerformanceRating', 'Count']
# Convert rating index to string to avoid Excel misinterpretation if it's mixed type initially
performance_rating_counts['PerformanceRating'] = performance_rating_counts['PerformanceRating'].astype(str)


# --- Visualization ---

# Apply custom style guidelines
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.edgecolor'] = '#1a1a24'
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['axes.grid'] = False # Turn off default grid
sns.set_theme(style="whitegrid") # Use a seaborn theme base, then apply customs


# 1. Distribution of Annual Salary (Histogram)
fig1, ax1 = plt.subplots(figsize=(10, 6))
# Use the numeric column, drop NaNs for plotting
sns.histplot(df['AnnualSalary'].dropna(), kde=True, ax=ax1, color='#188ce5') # Blue

ax1.set_title('Distribution of Annual Salary', fontsize=14, color='#1a1a24')
ax1.set_xlabel('Annual Salary', fontsize=12, color='#1a1a24')
ax1.set_ylabel('Frequency', fontsize=12, color='#1a1a24')
sns.despine(ax=ax1, top=True, right=True)
ax1.grid(False)

# Add mean/median lines if desired (optional, can clutter plot)
# ax1.axvline(df['AnnualSalary'].mean(), color='red', linestyle='dashed', linewidth=1, label='Mean')
# ax1.axvline(df['AnnualSalary'].median(), color='green', linestyle='dashed', linewidth=1, label='Median')
# ax1.legend()

# Format x-axis labels for currency/large numbers if needed
from matplotlib.ticker import FuncFormatter
def currency_formatter(x, pos):
    if x >= 1000000:
        return f'${x/1000000:,.1f}M'
    elif x >= 1000:
        return f'${x/1000:,.0f}K'
    else:
        return f'${x:,.0f}'

ax1.xaxis.set_major_formatter(FuncFormatter(currency_formatter))


plt.tight_layout()


# 2. Distribution of Department (Bar Chart)
fig2, ax2 = plt.subplots(figsize=(10, 6))
# Using a categorical color palette
colors = ['#ffe600', '#188ce5', '#2db757', '#ff6d00', '#750e5c', '#ff4136', '#27acaa', '#1a1a24'] # Colors from guidelines

# Use seaborn countplot directly or barplot on value_counts
sns.countplot(y='Department', data=df, order=df['Department'].value_counts().index, ax=ax2, palette=colors[:len(df['Department'].unique())])

ax2.set_title('Distribution of Employees by Department', fontsize=14, color='#1a1a24')
ax2.set_xlabel('Number of Employees', fontsize=12, color='#1a1a24')
ax2.set_ylabel('Department', fontsize=12, color='#1a1a24')
sns.despine(ax=ax2, top=True, right=True)
ax2.grid(False)

# Add data labels (counts)
for container in ax2.containers:
    ax2.bar_label(container, fmt='{:,.0f}', color='#1a1a24')

plt.tight_layout()


# Output results
# Return a dictionary containing the summary DataFrames and plots
output = {
    'Descriptive Summary (Numeric)': descriptive_summary_numeric,
    'Descriptive Summary (All Columns)': descriptive_summary_all,
    'Specific Calculated Metrics': specific_metrics,
    'Department Frequency Counts': department_counts,
    'Performance Rating Frequency Counts': performance_rating_counts,
    'Annual_Salary_Distribution_Plot': fig1,
    'Department_Distribution_Plot': fig2,
}

output # Output the dictionary

**Explanation:**

*   We load the dummy employee data. **Remember to replace `"EmployeeData"`**.
*   We explicitly convert columns to appropriate data types using `pd.to_numeric` with `errors='coerce'` to handle any non-numeric entries gracefully (turning them into `NaN`). We convert 'IsManager' to a nullable boolean.
*   `df.describe()` calculates standard descriptive statistics for numerical columns (count, mean, std, min, max, quartiles). Using `include='all'` also provides counts, unique values, top value, and frequency for object (string) and other types.
*   We calculate some specific metrics (mean, median, mode, standard deviation, variance, range, quartiles) using individual pandas series methods.
*   `value_counts()` is used to get the frequency distribution for categorical columns like 'Department' and 'PerformanceRating'. `dropna=False` includes missing values in the count.
*   **Visualization:**
    *   `fig1`: A histogram with a Kernel Density Estimate (KDE) curve shows the distribution of `AnnualSalary`, using `seaborn`. This helps visualize the shape of the distribution and identify potential skewness or outliers.
    *   `fig2`: A horizontal bar chart shows the count of employees in each `Department`, using `seaborn`. This clearly displays the size of each category.
*   **Custom Style:** Applied the specified style guidelines (font, colors - blue for histogram, palette for bars, off-black for text/axes, axes, spines, grid, data labels for the bar chart, currency formatting for the salary axis).
*   We return a dictionary containing the various summary DataFrames and the two plot figures.

**Viewing the Output:**

*   Click the Python cell, then click the Python icon/button next to the formula bar.
*   Select "Excel Value" (**Ctrl+Shift+Alt+M**) for the DataFrames ('Descriptive Summary (Numeric)', 'Descriptive Summary (All Columns)', 'Specific Calculated Metrics', 'Department Frequency Counts', 'Performance Rating Frequency Counts') to spill them into your sheet.
*   For each plot figure object ('Annual_Salary_Distribution_Plot', 'Department_Distribution_Plot'), select "Picture in Cell" > "Create Reference" to see the plots.

These descriptive statistics and visualizations provide a comprehensive overview of the dataset's key characteristics and distributions. The next topic in the series is ["Statistical Analysis - 2. Inferential Statistics"](./04-Statistical%20Analysis_02-Inferential%20Statistics.md), which builds upon these descriptive insights to make predictions and test hypotheses about the broader population.

**Further Analysis:**
* **Robust Statistics:** Implementing outlier-resistant statistics like median absolute deviation (MAD) and Huber's M-estimators
* **Distribution Fitting:** Using scipy to fit theoretical distributions and assess goodness-of-fit
* **Multivariate Analysis:** Calculating and visualizing correlations, covariances, and joint distributions
* **Bootstrap Statistics:** Implementing resampling methods to estimate sampling distributions and confidence intervals
* **Non-parametric Descriptives:** Using quartile-based statistics and rank-based measures for non-normal distributions